In [44]:
import re
import time
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from fuzzymatcher import link_table, fuzzy_left_join

In [45]:

import fresh_data.get_datasets
import importlib
importlib.reload(fresh_data.get_datasets)
from fresh_data.get_datasets import *

https://www.fec.gov/campaign-finance-data/congressional-candidate-data-summary-tables/?year=2024&segment=6

covering 01-01-1989 through 12-31-1990 two year periods



https://www.fec.gov/help-candidates-and-committees/making-disbursements/
disbursements

https://www.fec.gov/help-candidates-and-committees/keeping-records/recording-receipts/
receipts


In [46]:
fec = load_FEC_data("FEC/")
fec.head()

,year_range,state_name,district_code,representative,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts,congress
0,1989-1991,Alabama,1,"CALLAHAN, HERBERT L",Republican Party,Incumbent,318680.0,129153.0,169350.0,0.0,183910.0,236511.0,0.0,101
1,1989-1991,Alabama,2,"DICKINSON, WILLIAM LOUIS",Republican Party,Incumbent,425127.0,171618.0,218921.0,0.0,596096.0,250325.0,0.0,101
2,1989-1991,Alabama,2,"BAGGIANO, FAYE S",Democratic Party,Challenger,169109.0,38099.0,91752.0,20993.0,163663.0,5445.0,0.0,101
3,1989-1991,Alabama,3,"ADAMS, CHARLES H",Democratic Party,Open,277954.0,122590.0,19400.0,0.0,276975.0,978.0,55000.0,101
4,1989-1991,Alabama,3,"DIAL, GERALD OSCAR",Democratic Party,Open,111460.0,22420.0,14750.0,74290.0,111459.0,0.0,37894.0,101


In [47]:
fec.congress.unique()

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116])

In [48]:
fec.year_range.unique()

array(['1989-1991', '1991-1993', '1993-1995', '1995-1997', '1997-1999',
       '1999-2001', '2001-2003', '2003-2005', '2005-2007', '2007-2009',
       '2009-2011', '2011-2013', '2013-2015', '2015-2017', '2017-2019',
       '2019-2021'], dtype=object)

In [49]:
rep_mask = fec["representative"].apply(lambda rep: "LEVIN".lower() in rep.lower()  and ("Sander".lower() in rep.lower()))
fec[rep_mask]

,year_range,state_name,district_code,representative,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts,congress
1985,1991-1993,Michigan,17,"LEVIN, SANDER",Democratic Party,Incumbent,1028481.00,454110.00,504521.00,0.0,1185400.00,98288.00,0.0,102
3477,1993-1995,Michigan,12,"LEVIN, SANDER",Democratic Party,Incumbent,1462567.00,732040.00,687318.00,0.0,1539979.00,20889.00,36084.0,103
4951,1995-1997,Michigan,12,"LEVIN, SANDER",Democratic Party,Incumbent,1329479.00,704995.00,566482.00,0.0,1313913.00,36067.00,0.0,104
6192,1997-1999,Michigan,12,"LEVIN, SANDER M",Democratic Party,Incumbent,1787434.00,968637.00,776421.00,0.0,2058534.00,12765.00,64821.0,105
7348,1999-2001,Michigan,12,"LEVIN, SANDER M",Democratic Party,Incumbent,1261173.00,583979.00,667375.00,0.0,1481267.00,104447.00,3000.0,106
8632,2001-2003,Michigan,12,"LEVIN, SANDER",Democratic Party,Incumbent,941802.09,428488.30,497355.96,0.0,1007806.74,38442.61,0.0,107
9782,2003-2005,Michigan,12,"LEVIN, SANDER",Democratic Party,Incumbent,980161.27,443182.55,530196.07,0.0,869450.84,149153.04,0.0,108
11080,2005-2007,Michigan,12,"LEVIN, SANDER M",Democratic Party,Incumbent,626417.60,185608.20,439387.82,0.0,742098.22,33472.42,0.0,109
12436,2007-2009,Michigan,12,"LEVIN, SANDER M MR",Democratic Party,Incumbent,725442.68,168293.02,555295.00,0.0,660714.19,98200.91,0.0,110
14088,2009-2011,Michigan,12,"LEVIN, SANDER M MR",Democratic Party,Incumbent,2345154.74,946810.78,1394016.34,0.0,2392308.92,51046.73,0.0,111


Capps, Lois reps starting 106 but we don't want to include too much?


### Missing entries:

Duke Cunningham has FEC data from 101-107 but 105 is missing...

Pombo, Richard W has FEC data from 101-111 but 105 is missing

Maloney, James has FEC data from 103-114 but 105 is missing

FIELDS, Jack Milton, Jr. has FEC data for 101-102 but not for 104 ?

SCHUMER, Charles Ellis (Chuck) has data for 101-103 but not for 105?

NEUMANN, Mark W. has data for 102, 103, and 116, but not 105?



### Not in scope:

The following ran for 101st session of congress but the FEC didn't have data from before the 101st session from their campaigns:

FLIPPO, Ronnie Gene 

AKAKA, Daniel Kahikina

SAIKI, Patricia

MARTIN, Lynn Morley

SCHUETTE, Bill

FRENZEL, William Eldridge

DeWINE, Michael






In [50]:
fec.congress.unique()

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116])

In [51]:

for year_range in fec.year_range.unique():
    print(year_range, fec[fec["year_range"]==year_range].iloc[0]["congress"])

1989-1991 101
1991-1993 102
1993-1995 103
1995-1997 104
1997-1999 105
1999-2001 106
2001-2003 107
2003-2005 108
2005-2007 109
2007-2009 110
2009-2011 111
2011-2013 112
2013-2015 113
2015-2017 114
2017-2019 115
2019-2021 116
